# <center> <span style="font-family: Virgil GS, sans-serif; color:#97f788">XGBoost Scorecard Constructor</span> </center>
## <center> <span style="font-family: Virgil GS, sans-serif; color:navyblue">v1.0</span> </center>

 <span style="font-family: Virgil GS, sans-serif; color:navyblue">Author: <a href="https://github.com/deburky" title="GitHub link">https://github.com/deburky</a></span>


In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Generate a synthetic dataset
X, y = make_classification(
    n_samples=50000, n_features=15, n_classes=2, n_informative=5, random_state=42
)

X, y = pd.DataFrame(X), pd.DataFrame(y)

X.columns = [f"feature_{i+1}" for i in range(len(X.columns))]

ix_train, ix_test = train_test_split(X.index, stratify=y, random_state=62)

In [2]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score

# Create an XGBoost model
xgb_model = xgb.XGBClassifier(
    n_estimators=3,
    max_depth=2,
    random_state=42,
    early_stopping_rounds=3,
)

evalset = [(X.loc[ix_train], y.loc[ix_train]), (X.loc[ix_test], y.loc[ix_test])] # type: ignore

# Fit the model to the generated dataset
xgb_model.fit(X.loc[ix_train], y.loc[ix_train], eval_set=evalset) # type: ignore

# Make predictions using the model
predictions_trn = xgb_model.predict_proba(X.loc[ix_train])[:, 1] # type: ignore
predictions_tst = xgb_model.predict_proba(X.loc[ix_test])[:, 1] # type: ignore

gini_trn = roc_auc_score(y.loc[ix_train], predictions_trn) * 2 - 1 # type: ignore
gini_tst = roc_auc_score(y.loc[ix_test], predictions_tst) * 2 - 1 # type: ignore

print(f"Train Gini score: {gini_trn:.2%}\n" f"Test Gini score: {gini_tst:.2%}")

[0]	validation_0-logloss:0.59987	validation_1-logloss:0.59839
[1]	validation_0-logloss:0.54884	validation_1-logloss:0.54808
[2]	validation_0-logloss:0.51648	validation_1-logloss:0.51597
Train Gini score: 74.20%
Test Gini score: 74.11%


In [3]:
from xgb_scorecard_constructor import XGBScorecardConstructor
scorecard_constructor = XGBScorecardConstructor(xgb_model, X.loc[ix_train], y.loc[ix_train])
xgb_scorecard = scorecard_constructor.construct_scorecard()
xgb_scorecard

,Tree,Node,Feature,Sign,Split,Events,NonEvents,Total,EventRate,XAddEvidence,WOE
0,0,3,feature_14,<,-0.614810,4093.0,737.0,4830.0,0.847412,0.417061,-1.716152
1,0,4,feature_14,>=,-0.614810,5661.0,16309.0,21970.0,0.257670,-0.290232,1.056410
2,0,5,feature_2,<,-0.956827,502.0,825.0,1327.0,0.378297,-0.145094,0.495077
3,0,6,feature_2,>=,-0.956827,8478.0,895.0,9373.0,0.904513,0.485721,-2.250113
4,1,3,feature_13,<,-0.264047,3985.0,3132.0,7117.0,0.559927,0.076719,-0.242572
5,1,4,feature_13,>=,-0.264047,7850.0,1004.0,8854.0,0.886605,0.389491,-2.058228
6,1,5,feature_13,<,0.993707,4796.0,14233.0,19029.0,0.252036,-0.245816,1.086074
7,1,6,feature_13,>=,0.993707,2103.0,397.0,2500.0,0.841200,0.322957,-1.668891
8,2,3,feature_13,<,0.029582,6133.0,6241.0,12374.0,0.495636,0.019174,0.015750
9,2,4,feature_13,>=,0.029582,8669.0,1216.0,9885.0,0.876985,0.300165,-1.965893
